# Data Generation and Insertion for Transport Database

This notebook replaces the old SQL insert scripts with Python scripts for generating and inserting synthetic data into the database, using the new English schema. It uses Faker and pandas for data generation and manipulation.

In [1]:
import faker
import random
from datetime import datetime, timedelta
import os

# Initialize Faker for Colombian Spanish (to keep proper nouns in Spanish)
# Using 'es_CO' for names, cities, addresses if bogota_address is not specific enough
fake_co = faker.Faker('es_CO')
# For more diverse international-looking names if needed for some users, though 'es_CO' is primary
fake_generic = faker.Faker() # Can add other locales like fake_en = faker.Faker('en_US')


def generate_bogota_address():
    """Generates a more plausible Bogota-style address."""
    street_type = random.choice(["Calle", "Carrera", "Avenida", "Transversal", "Diagonal"])
    street_number = random.randint(1, 200)
    
    # Primary number, letter (optional), secondary number, complement (optional)
    part1 = random.randint(1, 150)
    part1_letter = random.choice(["", "A", "B", "C", "Bis"]) if random.random() > 0.5 else ""
    part2 = random.randint(1, 99)
    part3_complement = random.choice(["", f" Interior {random.randint(1,10)}", f" Apartamento {random.randint(100,1000)}", f" Oficina {random.randint(10,50)}"]) if random.random() > 0.7 else ""
    
    address_detail = f"{part1}{part1_letter} # {part2}-{random.randint(1,50)}"
    
    # Common neighborhoods in Bogota for added realism if desired, though Faker's city might be enough
    # neighborhoods = ["Chapinero", "Usaquén", "Suba", "Engativá", "Fontibón", "Kennedy", "Bosa", "Ciudad Bolívar", "Teusaquillo", "Barrios Unidos"]
    # neighborhood_detail = f", {random.choice(neighborhoods)}" if random.random() > 0.5 else ""
    
    return f"{street_type} {street_number} {address_detail}"


# --- Configuration ---
output_folder = "generated_sql_scripts"
users_output_file = os.path.join(output_folder, "1_insert_users.sql")
cards_output_file = os.path.join(output_folder, "2_insert_cards.sql")

# Ensure output directory exists
os.makedirs(output_folder, exist_ok=True)

# Number of records - using ranges for realism
# Target ~2.5M users
num_users = random.randint(24500, 25500)

# Target ~2M active cards (some users might have more than one card over time, but let's start with one active per user mostly)
# Let's make it so that most users get one active card.
# Total cards will be slightly more to account for some inactive/lost cards.
num_total_cards = random.randint(23000, 24000)
min_active_cards_target = 20000


# --- Lists for Data Generation ---
genders_list = ['M', 'F', 'O'] # O for Other

# --- Generate Users ---
print(f"Generating {num_users} users...")
user_ids_generated = [] # To keep track of generated user_ids for card assignment

with open(users_output_file, 'w', encoding='utf-8') as file:
    file.write("INSERT INTO users (user_id, first_name, last_name, contact_number, email, gender, date_of_birth, residential_address, id_number, city_of_birth, registration_date) VALUES\n")

    batch_size = 1000 # Insert 1000 records at a time then restart VALUES
    for i in range(num_users):
        user_id = i + 1 # Assuming user_id starts from 1 and is sequential for this script
        user_ids_generated.append(user_id)

        # Mix of Colombian and more generic names for wider appearance
        if random.random() < 0.85: # 85% Colombian-style names
            first_name_val = fake_co.first_name().replace("'", "''")
            last_name_val = fake_co.last_name().replace("'", "''")
        else:
            first_name_val = fake_generic.first_name().replace("'", "''")
            last_name_val = fake_generic.last_name().replace("'", "''")
            
        contact_number_val = fake_co.phone_number() # Colombian format
        email_val = fake_co.unique.email() # Ensure unique email
        gender_val = random.choice(genders_list)
        
        birth_date_obj = fake_co.date_of_birth(minimum_age=16, maximum_age=85)
        date_of_birth_val = birth_date_obj.strftime('%Y-%m-%d')
        
        residential_address_val = generate_bogota_address().replace("'", "''")
        # Generate a unique national ID number (Cédula)
        id_number_val = str(fake_co.unique.random_number(digits=10, fix_len=True))
        
        city_of_birth_val = fake_co.city().replace("'", "''") # Colombian city
        
        registration_date_obj = fake_co.date_between(start_date='-10y', end_date='today')
        registration_date_val = registration_date_obj.strftime('%Y-%m-%d')
        
        # SQL formatting
        file.write(f"({user_id}, '{first_name_val}', '{last_name_val}', '{contact_number_val}', '{email_val}', '{gender_val}', '{date_of_birth_val}', '{residential_address_val}', '{id_number_val}', '{city_of_birth_val}', '{registration_date_val}')")

        if (i + 1) % batch_size == 0 and i < num_users -1:
            file.write(";\n")
            file.write("INSERT INTO users (user_id, first_name, last_name, contact_number, email, gender, date_of_birth, residential_address, id_number, city_of_birth, registration_date) VALUES\n")
        elif i < num_users - 1:
            file.write(",\n")
        else:
            file.write(";\n")
            
print(f"SQL script for users generated: {os.path.abspath(users_output_file)}")


# --- Generate Cards ---
print(f"Generating {num_total_cards} cards (aiming for at least {min_active_cards_target} active)...")
active_cards_count = 0
card_statuses = ['active', 'inactive', 'blocked', 'lost']

with open(cards_output_file, 'w', encoding='utf-8') as file:
    file.write("INSERT INTO cards (card_id, card_number, user_id, acquisition_date, status, balance, last_used_date, update_date) VALUES\n")
    
    assigned_users_for_cards = set() # To ensure a user gets at least one card if possible

    for i in range(num_total_cards):
        card_id = i + 1 # Assuming card_id starts from 1
        
        # Generate a unique card number (can be more complex if needed)
        card_number_val = str(fake_co.unique.random_number(digits=16, fix_len=True))
        
        user_id_val = None
        # Try to assign to a user who doesn't have many cards yet or ensure enough users get one
        if user_ids_generated:
            if len(assigned_users_for_cards) < len(user_ids_generated) and active_cards_count < min_active_cards_target :
                 # Prioritize users who haven't been assigned a card yet for active cards
                potential_users = [uid for uid in user_ids_generated if uid not in assigned_users_for_cards]
                if potential_users:
                    user_id_val = random.choice(potential_users)
                    assigned_users_for_cards.add(user_id_val)
                else: # all users got one, assign randomly
                    user_id_val = random.choice(user_ids_generated)
            else: # Random assignment after targets are met or if all users have one
                 user_id_val = random.choice(user_ids_generated)


        if user_id_val is None: # Fallback if no users somehow (should not happen with num_users > 0)
            user_id_val = "NULL"


        acquisition_date_obj = fake_co.date_between(start_date='-8y', end_date='today') # Card acquired after user registration potentially
        acquisition_date_val = acquisition_date_obj.strftime('%Y-%m-%d')
        
        status_val = 'inactive' # Default
        if active_cards_count < min_active_cards_target:
            # Higher chance of being active until target is met
            status_val = random.choices(card_statuses, weights=[0.9, 0.05, 0.03, 0.02], k=1)[0]
        else:
            # Normal distribution after target
            status_val = random.choices(card_statuses, weights=[0.7, 0.15, 0.1, 0.05], k=1)[0]
        
        if status_val == 'active':
            active_cards_count += 1
            
        balance_val = 0.0
        if status_val == 'active' and random.random() < 0.8: # 80% of active cards have some balance
            balance_val = round(random.uniform(1000, 50000) / 50) * 50 # Multiples of 50 COP
            
        last_used_date_val = "NULL"
        if status_val == 'active' and random.random() < 0.9: # 90% of active cards have been used
            last_used_datetime_obj = fake_co.date_time_between(start_date=acquisition_date_obj, end_date='now', tzinfo=None)
            last_used_date_val = f"'{last_used_datetime_obj.strftime('%Y-%m-%d %H:%M:%S')}'"
            
        update_date_obj = fake_co.date_between(start_date=acquisition_date_obj, end_date='today')
        update_date_val = update_date_obj.strftime('%Y-%m-%d')

        file.write(f"({card_id}, '{card_number_val}', {user_id_val}, '{acquisition_date_val}', '{status_val}', {balance_val}, {last_used_date_val}, '{update_date_val}')")

        if (i + 1) % batch_size == 0 and i < num_total_cards -1:
            file.write(";\n")
            file.write("INSERT INTO cards (card_id, card_number, user_id, acquisition_date, status, balance, last_used_date, update_date) VALUES\n")
        elif i < num_total_cards - 1:
            file.write(",\n")
        else:
            file.write(";\n")

print(f"SQL script for cards generated: {os.path.abspath(cards_output_file)}")
print(f"Total active cards generated: {active_cards_count}")

Generating 25166 users...
SQL script for users generated: /home/kali/Documents/remote reps/distributed-systems-lab/travel-recharge-database/db/data/generated_sql_scripts/1_insert_users.sql
Generating 23202 cards (aiming for at least 20000 active)...
SQL script for cards generated: /home/kali/Documents/remote reps/distributed-systems-lab/travel-recharge-database/db/data/generated_sql_scripts/2_insert_cards.sql
Total active cards generated: 20643


In [2]:
import faker
import random
from datetime import datetime
import os
import unicodedata

# Initialize Faker for Colombian Spanish
fake_co = faker.Faker('es_CO')

# --- Configuration ---
output_folder = "generated_sql_scripts" # Defined in the previous script
locations_output_file = os.path.join(output_folder, "3_insert_locations.sql")
recharge_points_output_file = os.path.join(output_folder, "4_insert_recharge_points.sql")

# Ensure output directory exists (though previous script should have created it)
os.makedirs(output_folder, exist_ok=True)

# Number of records
num_locations = 13 # Based on "13 Zonas de Operación" [cite: 28]
# Based on "4,864 puntos de recarga externos" [cite: 28]
num_recharge_points = random.randint(4800, 4900)

# Function to generate a more plausible Bogota-style address (can be shared across scripts)
def generate_bogota_address_simple():
    """Generates a simplified plausible Bogota-style address for recharge points."""
    street_type = random.choice(["Calle", "Carrera", "Avenida", "Transversal", "Diagonal"])
    street_number = random.randint(1, 200)
    part1 = random.randint(1, 150)
    part2 = random.randint(1, 99)
    address_detail = f"{part1} # {part2}-{random.randint(1,50)}"
    return f"{street_type} {street_number} {address_detail}"

def slugify(text):
    """Converts text to a simple slug."""
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('ascii')
    text = text.lower().replace(' ', '_').replace('.', '').replace(',', '')
    return text


# --- Generate Locations (Zonas de Operación) ---
print(f"Generating {num_locations} locations (operational zones)...")

# Names based on zones mentioned or implied in the document [cite: 25]
# Tunal, Sur, Américas, Calle 80, Norte, Suba, Usme, Engativá, San Cristóbal,
# Usaquén, Fontibón, Kennedy, Ciudad Bolívar. We need 13.
# Some from concessionaire zones[cite: 25]: Engativá, San Cristóbal, Usaquén, Calle 80, Tintal Zona Franca (use Fontibón for this), Bosa, Suba Oriental, Kennedy, Ciudad Bolívar, Fontibón, Usme, Suba Centro, Perdomo.
# Let's refine the list to 13 distinct major zones often referenced.
location_names_base = [
    "Usaquén", "Chapinero", "Santa Fe", "San Cristóbal", "Usme", "Tunjuelito",
    "Bosa", "Kennedy", "Fontibón", "Engativá", "Suba", "Barrios Unidos", "Teusaquillo"
    # "Puente Aranda", "Los Mártires", "Antonio Nariño", "Ciudad Bolívar", "Sumapaz" # Other Localidades
]
if len(location_names_base) < num_locations:
    location_names_base.extend([f"Zona Operativa {i+1}" for i in range(num_locations - len(location_names_base))])
elif len(location_names_base) > num_locations:
    location_names_base = random.sample(location_names_base, num_locations)


location_ids_generated = []

with open(locations_output_file, 'w', encoding='utf-8') as file:
    file.write("INSERT INTO locations (location_id, name, description) VALUES\n")
    batch_size = 1000

    for i in range(num_locations):
        location_id = i + 1 # Assuming location_id starts from 1
        location_ids_generated.append(location_id)
        
        name_val = location_names_base[i].replace("'", "''")
        description_val = f"Zona de operación {name_val} en Bogotá.".replace("'", "''")
        
        file.write(f"({location_id}, '{name_val}', '{description_val}')")
        
        if (i + 1) % batch_size == 0 and i < num_locations -1 :
            file.write(";\n")
            file.write("INSERT INTO locations (location_id, name, description) VALUES\n")
        elif i < num_locations - 1:
            file.write(",\n")
        else:
            file.write(";\n")

print(f"SQL script for locations generated: {os.path.abspath(locations_output_file)}")


# --- Generate Recharge Points ---
print(f"Generating {num_recharge_points} recharge points...")

recharge_point_operators = ["PuntoRed", "SuRed", "MoviiRed", "PagaTodo", "Station Kiosk", "Online Platform"]

with open(recharge_points_output_file, 'w', encoding='utf-8') as file:
    file.write("INSERT INTO recharge_points (recharge_point_id, name, address, latitude, longitude, location_id, operator) VALUES\n")
    batch_size = 1000

    for i in range(num_recharge_points):
        recharge_point_id = i + 1 # Assuming recharge_point_id starts from 1
        
        # Generate a plausible name for the recharge point
        point_type = random.choice(["Tienda", "Papelería", "Droguería", "Miscelánea", "Kiosko Estación", "Plataforma Web"])
        base_name_for_point = fake_co.company().split(' ')[0].replace(',', '') + " " + fake_co.street_name().split(' ')[-1]
        name_val = f"{point_type} {base_name_for_point}".replace("'", "''")
        if "Plataforma Web" in name_val:
            name_val = "Plataforma de Recarga Online TransMilenio" # Make it more unique if it's an online platform
            address_val = "NULL" # No physical address for online
            # Bogota's general coordinates for online services or use NULL
            latitude_val = "NULL"
            longitude_val = "NULL"
            operator_val = "Online Platform"
        else:
            address_val = f"'{generate_bogota_address_simple().replace("'", "''")}'"
            # Generate Lat/Long for Bogotá (approximate bounds)
            # Lat: 4.4 to 4.8, Lon: -74.0 to -74.2
            latitude_val = round(random.uniform(4.400000, 4.800000), 6)
            longitude_val = round(random.uniform(-74.200000, -74.000000), 6)
            operator_val = random.choice(recharge_point_operators)


        location_id_val = random.choice(location_ids_generated) if location_ids_generated else "NULL"
        
        file.write(f"({recharge_point_id}, '{name_val}', {address_val}, {latitude_val}, {longitude_val}, {location_id_val}, '{operator_val}')")

        if (i + 1) % batch_size == 0 and i < num_recharge_points -1 :
            file.write(";\n")
            file.write("INSERT INTO recharge_points (recharge_point_id, name, address, latitude, longitude, location_id, operator) VALUES\n")
        elif i < num_recharge_points - 1:
            file.write(",\n")
        else:
            file.write(";\n")
            
print(f"SQL script for recharge points generated: {os.path.abspath(recharge_points_output_file)}")

Generating 13 locations (operational zones)...
SQL script for locations generated: /home/kali/Documents/remote reps/distributed-systems-lab/travel-recharge-database/db/data/generated_sql_scripts/3_insert_locations.sql
Generating 4834 recharge points...
SQL script for recharge points generated: /home/kali/Documents/remote reps/distributed-systems-lab/travel-recharge-database/db/data/generated_sql_scripts/4_insert_recharge_points.sql


In [ ]:
import faker
import random
from datetime import datetime, timedelta
import os
import uuid

# Initialize Faker for Colombian Spanish
fake_co = faker.Faker('es_CO')

# --- Configuration ---
output_folder = "generated_sql_scripts" # Defined in previous scripts
recharges_output_file = os.path.join(output_folder, "5_insert_recharges.sql")

# Ensure output directory exists
os.makedirs(output_folder, exist_ok=True)

# Max IDs from previous scripts (using upper end of their generation ranges)
# From 2_insert_cards.sql: num_total_cards = random.randint(2300000, 2400000)
max_card_id = 24000
# From 2_insert_cards.sql: min_active_cards_target = 2000000 (these cards should see more activity)
assumed_active_card_threshold_id = 20000

# From 4_insert_recharge_points.sql: num_recharge_points = random.randint(4800, 4900)
max_recharge_point_id = 4900
# Assume one of the recharge points was the "Online Platform" as per logic in 4_insert_recharge_points
# For simplicity, let's assume the last ID could be an online one, or a specific known ID if designated.
# For now, we'll treat all physical points similarly popular, with online being an option.

# Number of recharge records
num_recharges = random.randint(65000, 80000) # e.g., 7 million recharges (real file estimate)

# Common recharge amounts in COP
recharge_amounts_cop = [
    3000, 3200, 3950, 7000, 10000, 11800, 15000, 20000, 23600, 30000, 70000, 50000, 100000
]
# Weights for these amounts (e.g., 10000 and 20000 are very common)
recharge_amounts_weights = [
    10, 5, 10, 20, 5, 25, 10, 5, 20, 5, 3, 10, 2
]


# --- Generate Recharges ---
print(f"Generating {num_recharges} recharge records...")

with open(recharges_output_file, 'w', encoding='utf-8') as file:
    file.write("INSERT INTO recharges (recharge_id, card_id, recharge_point_id, amount, recharge_timestamp, transaction_id) VALUES\n")
    
    batch_size = 1000 # SQL statements per batch

    for i in range(num_recharges):
        recharge_id = i + 1 # Assuming recharge_id starts from 1

        # Select card_id: Prioritize "active" cards
        if random.random() < 0.85 and assumed_active_card_threshold_id > 0 : # 85% of recharges go to presumed active cards
            card_id_val = random.randint(1, assumed_active_card_threshold_id)
        else:
            card_id_val = random.randint(1, max_card_id)

        # Select recharge_point_id
        # Give a slight preference to non-online points for more "physical" transactions
        if random.random() < 0.05: # 5% chance of using the "Online Platform" (assuming it's the last ID or a known one)
            # If the online platform was the last ID in recharge_points script:
            recharge_point_id_val = max_recharge_point_id
            # Or if it had a specific name/ID, you'd target that. For now, last ID is a placeholder.
        else:
            recharge_point_id_val = random.randint(1, max_recharge_point_id -1 if max_recharge_point_id > 1 else 1)


        amount_val = random.choices(recharge_amounts_cop, weights=recharge_amounts_weights, k=1)[0]

        # Simulate card acquisition date (consistent with card generation logic)
        # Cards are acquired between 8 years ago and today.
        # A card's recharges must happen after its acquisition.
        # For simplicity, we simulate an acquisition window for each recharge event.
        # A more complex approach would be to pre-generate acquisition dates for all cards.
        # Here, we assume older cards (lower IDs) could have been acquired earlier.
        years_ago_for_card = min(8, max(1, int(8 * (card_id_val / max_card_id)))) # Rough estimate: newer cards acquired more recently
        
        try:
            # Simulate a plausible acquisition date for this specific card_id_val
            # To ensure recharge_timestamp is after this.
            simulated_acquisition_date = datetime.now() - timedelta(days=random.randint(30, years_ago_for_card * 365))
            if simulated_acquisition_date > datetime.now() - timedelta(days=30): # Ensure it's at least 30 days old
                simulated_acquisition_date = datetime.now() - timedelta(days=30)

            recharge_datetime_obj = fake_co.date_time_between(start_date=simulated_acquisition_date, end_date='now', tzinfo=None)
        except: # Fallback if date ranges are problematic
             recharge_datetime_obj = fake_co.date_time_between(start_date='-5y', end_date='now', tzinfo=None)

        recharge_timestamp_val = recharge_datetime_obj.strftime('%Y-%m-%d %H:%M:%S')
        
        transaction_id_val = str(uuid.uuid4())

        # SQL formatting
        file.write(f"({recharge_id}, {card_id_val}, {recharge_point_id_val}, {amount_val}, '{recharge_timestamp_val}', '{transaction_id_val}')")

        if (i + 1) % batch_size == 0 and i < num_recharges - 1:
            file.write(";\n")
            file.write("INSERT INTO recharges (recharge_id, card_id, recharge_point_id, amount, recharge_timestamp, transaction_id) VALUES\n")
        elif i < num_recharges - 1:
            file.write(",\n")
        else:
            file.write(";\n")
            
print(f"SQL script for recharges generated: {os.path.abspath(recharges_output_file)}")

Generating 70696 recharge records...
SQL script for recharges generated: /home/kali/Documents/remote reps/distributed-systems-lab/travel-recharge-database/db/data/generated_sql_scripts/5_insert_recharges.sql
